参考资料1：[Parallel programming in Python: multiprocessing (part 1)
](https://www.kth.se/blogs/pdc/2019/02/parallel-programming-in-python-multiprocessing-part-1/)           
参考资料2：[Parallel programming in Python: multiprocessing (part 2)
](https://www.kth.se/blogs/pdc/2019/02/parallel-programming-in-python-multiprocessing-part-2/) 

In [19]:
import multiprocessing as mp
from tqdm.notebook import tqdm

In [20]:
#org_list = list(range(20))
org_list = [3,4,5,6,5,4,3,2,1,3,4,4,3,2,4,6,2,1,32,4,55,3,2,1,4,6,78,8,5,6,0,9,8,6,3,5,6,7,7,8,8,4,2]

# Single Processing

In [21]:
%%time
res1 = []
res2 = []
j = 0

for i in tqdm(org_list):

  res1.append(i**2)
  res1.append(i**10)


CPU times: user 54.4 ms, sys: 7.69 ms, total: 62.1 ms
Wall time: 95.5 ms


# Multi-processing 1 傻瓜版
## Synchronous method

- `map/imap`: single argument
- `starmap`: multiple argument

Note that both `map` and `starmap` are **synchronous** methods. In other words, if a worker process finishes its sub-task very early, it will wait for the other worker processes to finish. This may lead to performance degradation if the workload is not well balanced among the worker processes.

In [22]:
import multiprocessing

#cores = multiprocessing.cpu_count()  # Count the number of cores in a computer
pool = multiprocessing.Pool(processes=2) 

def func(item):
  # res1 = []
  # res2 = []

  # res1.append(item**2)
  # res1.append(item**10)

    return [item**2, item**10]

In [23]:
pool = multiprocessing.Pool(processes=2)
sim_lst1 = list(tqdm(pool.imap(func, org_list)))
    
pool.close()
pool.join()

In [24]:
pool = multiprocessing.Pool(processes=2)
sim_lst2 = list(tqdm(pool.map(func, org_list)))
    
pool.close()
pool.join()

In [25]:
sim_lst3 = list(tqdm(map(func, org_list)))

In [26]:
sim_lst2 == sim_lst3

True

In [27]:
def power_n(x, n):
    return x ** n

In [28]:
pool = multiprocessing.Pool(processes=2)

result = pool.starmap(power_n, [(x, 2) for x in range(20)])
print(result)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]


## Asynchronous method

- `apply_async`

The Pool class also provides the `apply_async` method that makes **asynchronous** execution of the worker processes possible. Unlike the `map` method, which executes a computational routine over a list of inputs, the `apply_async` method executes the routine only once. Therefore, in the previous example, we would need to define another routine, `power_n_list`, that computes the values of a list of numbers raised to a particular power.

In [29]:
nsteps = 10000000
dx = 1.0 / nsteps
pi = 0.0

In [30]:
for i in range(nsteps):
    x = (i + 0.5) * dx
    pi += 4.0 / (1.0 + x * x)
pi *= dx

In [31]:
def calc_partial_pi(rank, nprocs, nsteps, dx):
    partial_pi = 0.0
    for i in range(rank, nsteps, nprocs):
        x = (i + 0.5) * dx
        partial_pi += 4.0 / (1.0 + x * x)
    partial_pi *= dx
    return partial_pi

In [32]:
import multiprocessing as mp

nprocs = mp.cpu_count()
inputs = [(rank, nprocs, nsteps, dx) for rank in range(nprocs)]

In [33]:
# starmap

pool = mp.Pool(processes=nprocs)
result = pool.starmap(calc_partial_pi, inputs)
pi = sum(result)

In [34]:
# apply_async

multi_result = [pool.apply_async(calc_partial_pi, inp) for inp in inputs]

# 运行下面这个list才会真的开始processing
result = [p.get() for p in multi_result]  
pi = sum(result)

## Summary

- `map` and `starmap` are synchronous methods.
- `map` and `starmap` guarantee the correct order of output.
- `starmap` and `apply_async` support multiple arguments.

# Multi-processing 2 进阶版

In the previous post we introduced the `Pool` class of the multiprocessing module. In this post we continue on and introduce the `Process` class, which makes it possible to have direct control over individual processes.   


采用`Process` class，需要自己chuck list，定义流程，不过更灵活，功能也更强大

- `Process`: 安排运算
- `Queue`: 保存运算结果 (`put`/`get`, The `Queue` class includes the `put` method for depositing data and the `get` method for retrieving data.)


样例代码(无序输出版):

```python
import multiprocessing as mp

def square(x, q):
    q.put(x * x)

qout = mp.Queue()
processes = [mp.Process(target=square, args=(i, qout))
             for i in range(2, 10)]

for p in processes:
    p.start()

for p in processes:
    p.join()

result = [qout.get() for p in processes]
print(result)

```


样例代码(有序输出版):

```python
import multiprocessing as mp
from random import randint
from time import sleep

def square(i, x, q):
    sleep(0.01 * randint(0, 100)) 
    q.put((i, x * x))

input_values = [2, 4, 6, 8, 3, 5, 7, 9]
qout = mp.Queue()
processes = [mp.Process(target=square, args=(ind, val, qout))
             for ind, val in enumerate(input_values)]

for p in processes:
    p.start()

for p in processes:
    p.join()

unsorted_result = [qout.get() for p in processes]
result = [t[1] for t in sorted(unsorted_result)] 
print(result)
```



先split原list成多个sublist：

In [35]:
def split(a, n):
  '''
  a: 需要chunk的large list
  n: 希望chunk出多少个sublist
  '''
  k, m = divmod(len(a), n)
  split_data = [a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]

  # 没办法，我们得在每个sublist前面做好index的标记，最后拼接result的时候会用到
  split_data_order_number = [[i, v] for i, v in enumerate(split_data)]

  return split_data_order_number
        
sub_list = split(org_list,4)

In [36]:
#org_list

In [37]:
list(sub_list)

[[0, [3, 4, 5, 6, 5, 4, 3, 2, 1, 3, 4]],
 [1, [4, 3, 2, 4, 6, 2, 1, 32, 4, 55, 3]],
 [2, [2, 1, 4, 6, 78, 8, 5, 6, 0, 9, 8]],
 [3, [6, 3, 5, 6, 7, 7, 8, 8, 4, 2]]]

In [38]:
%%time

def func(x, q1, q2, q3):
    
    
    index = x[0]
    value = x[1]
    res1 = []
    res2 = []
    
    print(f'Job {index} starting\n')
    for i in tqdm(value):

      res1.append(i**2)
      res2.append(i**3)
        
    # 我们不用return，而是q.put()来接收输出    
    q1.put(res1)
    q2.put(res2)
    q3.put(index)
    print(f'Job {index} finishing\n')

qout1 = mp.Queue()
qout2 = mp.Queue()
qout3 = mp.Queue()

# 把要处理的list放在`sub_list`
# 要注意的是，每个process结束，结果也会被存储，不一定是按照sub_list的顺序来存储的，谁先完成谁先存储
processes = [mp.Process(target=func, args=(i, qout1, qout2, qout3)) for i in sub_list] 

CPU times: user 1.92 ms, sys: 0 ns, total: 1.92 ms
Wall time: 2.26 ms


In [39]:
for p in processes:
    p.daemon = True
    p.start()
    

# 到下面`qout.get()`这一步，processing才正式开始
# 按照前面chunk的index进行排序
unsorted_result = [[qout1.get(), qout2.get(), qout3.get()] for p in processes]
result = sum([t[0] for t in sorted(unsorted_result, key=lambda x: x[2])], []) 


for p in processes:
    p.join()


print('All task is done!')

Job 0 starting
Job 1 starting
Job 2 starting



Job 3 starting




Job 0 finishing


Job 1 finishing


Job 3 finishing


Job 2 finishing

All task is done!


In [40]:
result == [i**2 for i in org_list]

True

Summary:

- The `Proces`s class makes it possible to control the processes directly.
- The `Queue` class can be used to save results from the processes.
- The processes are executed asynchronously.
- The order of the output is not guaranteed to correspond to that of the input values.